In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

C:\Users\sonya\AppData\Local\Temp\ipykernel_32608\3461523344.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
ruta_archivo = "../Datos/ObesityDataSet_raw_and_data_sinthetic.csv"

df = pd.read_csv(ruta_archivo)
# Eliminar las columnas "Height" y "Weight"
df = df.drop(["Height", "Weight"], axis=1)
df

,Gender,Age,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


In [3]:
# Definir la variable objetivo (y)
y = df["NObeyesdad"]

# Definir características (X) excluyendo la variable objetivo
X = df.drop("NObeyesdad", axis=1)

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Imprimir la forma de los conjuntos de entrenamiento y prueba para verificar
print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

Forma de X_train: (1688, 14)
Forma de X_test: (423, 14)
Forma de y_train: (1688,)
Forma de y_test: (423,)


In [4]:
# Definir el orden deseado de las categorías
orden_categorias = {
    'Insufficient_Weight': 1,
    'Normal_Weight': 2,
    'Overweight_Level_I': 3,
    'Overweight_Level_II': 4,
    'Obesity_Type_I': 5,
    'Obesity_Type_II': 6,
    'Obesity_Type_III': 7
}

# Codificar la variable objetivo y_train con el orden especificado
y_train = np.array([orden_categorias[cat] for cat in y_train])

In [5]:
print("Valores codificados de y:", y_train)

Valores codificados de y: [1 4 1 ... 6 4 5]


In [6]:
# Identificar las columnas numéricas en X_train
columnas_numericas = X_train.select_dtypes(include=['float64', 'int64']).columns

# Construir el pipeline para las características numéricas
pipeline_numerico = Pipeline([
    ('scaler', StandardScaler())
])

# Crear el transformador de columnas
transformador_columnas = ColumnTransformer([
    ('numeric', pipeline_numerico, columnas_numericas)
])

# Aplicar el transformador a X_train
X_train_procesado = transformador_columnas.fit_transform(X_train)

# Verificar la forma de X_train_procesado
print("Forma de X_train_procesado:", X_train_procesado.shape)

print("Componentes del Pipeline:")
for nombre, paso in transformador_columnas.named_transformers_.items():
    print(nombre)
    print(paso)

Forma de X_train_procesado: (1688, 6)
Componentes del Pipeline:
numeric
Pipeline(steps=[('scaler', StandardScaler())])
remainder
drop


In [7]:
# Establecer una semilla para reproducibilidad
np.random.seed(123)
# Convertir X_train a DataFrame de pandas
X_train_df = pd.DataFrame(X_train)

# Identificar las columnas numéricas en X_train
columnas_numericas = X_train_df.select_dtypes(include=['float64', 'int64']).columns
# Aplicar el transformador a X_test
X_test_procesado = transformador_columnas.transform(X_test)

# Definir los hiperparámetros a explorar
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Parámetro de regularización
    'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],  # Algoritmo de optimización
    'max_iter': [100, 200, 300]  # Máximo número de iteraciones
}

# Inicializar el modelo logístico multinomial
model = LogisticRegression(multi_class='multinomial')

# Realizar la búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_procesado, y_train)

# Obtener el mejor modelo
best_model = grid_search.best_estimator_

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Mostrar el mejor puntaje de validación cruzada
print("Mejor puntaje de validación cruzada:")
print(grid_search.best_score_)

# Evaluar el mejor modelo en el conjunto de prueba
accuracy = best_model.score(X_test_procesado, y_test)
#print("Precisión del mejor modelo en el conjunto de prueba:", accuracy)

c:\Users\sonya\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\sonya\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\sonya\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\sonya\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\sonya\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the 

Mejores hiperparámetros encontrados:
{'C': 100, 'max_iter': 100, 'solver': 'lbfgs'}
Mejor puntaje de validación cruzada:
0.3732094885256264


c:\Users\sonya\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [8]:
# Definir las columnas categóricas
categorical_columns = ['Gender']  # Se eliminó 'Education'

# Crear el transformador para las columnas categóricas
categorical_transformer = OneHotEncoder(drop='first')

# Crear el transformador de columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_columns)
    ],
    remainder='passthrough'
)

# Aplicar el preprocesamiento a los datos
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [10]:
# Establecer la semilla para reproducibilidad
np.random.seed(123)

# Definir las columnas numéricas
columnas_numericas = X_train.select_dtypes(include=['float64', 'int64']).columns

# Construir el pipeline para las características numéricas
pipeline_numerico = Pipeline([
    ('scaler', StandardScaler())
])

# Crear el transformador de columnas
transformador_columnas = ColumnTransformer([
    ('numeric', pipeline_numerico, columnas_numericas)
])

# Aplicar el transformador a X_train y X_test
X_train_procesado = transformador_columnas.fit_transform(X_train)
X_test_procesado = transformador_columnas.transform(X_test)

# Inicializar el modelo de Random Forest
random_forest = RandomForestClassifier(random_state=123)

# Entrenar el modelo
random_forest.fit(X_train_procesado, y_train)

# Predecir en el conjunto de prueba
y_pred = random_forest.predict(X_test_procesado)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
#print("Precisión del modelo de Random Forest:", accuracy)